In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
import os

In [6]:
nowpath=os.path.abspath('.')
t_log=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_login.csv',sep=',')
t_trade=pd.read_csv(nowpath+'/Risk_Detection_Qualification/t_trade.csv',sep=',')

In [7]:
pd.options.display.max_rows = 100

In [1]:
'''df_cleared=pd.DataFrame(data=t_log.drop('log_id',1).sort_values(by='time',axis=0))
df_cleared.index=range(len(df_cleared))
df_cleared['is_scan_num']=((df_cleared['is_scan']==True)).astype(int)
df_cleared['is_sec_num']=((df_cleared['is_sec']==True)).astype(int)
df_cleared=df_cleared.drop('is_scan',1)
df_cleared=df_cleared.drop('is_sec',1)
df_cleared'''

"df_cleared=pd.DataFrame(data=t_log.drop('log_id',1).sort_values(by='time',axis=0))\ndf_cleared.index=range(len(df_cleared))\ndf_cleared['is_scan_num']=((df_cleared['is_scan']==True)).astype(int)\ndf_cleared['is_sec_num']=((df_cleared['is_sec']==True)).astype(int)\ndf_cleared=df_cleared.drop('is_scan',1)\ndf_cleared=df_cleared.drop('is_sec',1)\ndf_cleared"

In [8]:
def JD_log_dataclear(df):
    df_cleared=pd.DataFrame(data=df.drop('log_id',1).sort_values(by='time',axis=0))
    df_cleared.index=range(len(df_cleared))
    df_cleared['is_scan_num']=((df_cleared['is_scan']==True)).astype(int)
    df_cleared['is_sec_num']=((df_cleared['is_sec']==True)).astype(int)
    df_cleared['time_pd']=[pd.Timestamp(df_cleared.loc[t,'time']) for t in range(len(df_cleared))]
    df_cleared=df_cleared.drop('is_scan',1)
    df_cleared=df_cleared.drop('is_sec',1)
    df_cleared=df_cleared.drop('time',1)
    return df_cleared

In [16]:
def JD_trade_dataclear(df):
    df_cleared=pd.DataFrame(data=df.sort_values(by='time',axis=0))
    df_cleared.index=range(len(df_cleared))
    df_cleared['time_pd']=[pd.Timestamp(df_cleared.loc[t,'time']) for t in range(len(df_cleared))]
    df_cleared=df_cleared.drop('time',1)
    df_cleared['weekdays']=[df_cleared.loc[t,'time_pd'].weekday() for t in range(len(df_cleared))]
    return df_cleared

In [10]:
df_log_use=JD_log_dataclear(t_log)

In [17]:
df_trade_use=JD_trade_dataclear(t_trade)

In [23]:
def JD_trade_merge(df_log_use,df_trade_use):
    df_trade_use['device']=0
    df_trade_use['log_from']=0
    df_trade_use['ip']=0
    df_trade_use['city']=0
    df_trade_use['result']=0
    df_trade_use['timestamp']=0
    df_trade_use['type']=0
    df_trade_use['is_scan_num']=0
    df_trade_use['is_sec_num']=0
    #df_trade_use=df_trade_use.drop(('device','log_from'),1)
    this_row='Null'
    nofit_count=0
    #nofit_danger_count=0
    df_log_use=df_log_use.sort_values(by='time_pd',axis=0,ascending=False)
    df_log_use.index=range(len(df_log_use))
    for i in range(len(df_trade_use)):
        df_tmp=df_log_use[df_log_use.loc[:,'id']==df_trade_use.loc[i,'id']]
        sigsig=False
        row='Null'
        #this_row='Null'
        for rowd in df_tmp.iterrows():
            row=rowd[1]
            #if df_trade_use.loc[i,'time_pd']-row['time_pd']<pd.Timedelta('01:00:00'):
            if df_trade_use.loc[i,'time_pd']<row['time_pd']:
                df_trade_use.loc[i,'device']=row['device']
                df_trade_use.loc[i,'log_from']=row['log_from']
                df_trade_use.loc[i,'ip']=row['ip']
                df_trade_use.loc[i,'city']=row['city']
                df_trade_use.loc[i,'result']=row['result']
                df_trade_use.loc[i,'timestamp']=row['timestamp']
                df_trade_use.loc[i,'type']=row['type']
                df_trade_use.loc[i,'is_scan_num']=row['is_scan_num']
                df_trade_use.loc[i,'is_sec_num']=row['is_sec_num']
                sigsig=True
                break
            #this_row=row.copy()
            #print(this_row)
        if sigsig==False:
            nofit_count=nofit_count+1
            df_trade_use=df_trade_use.drop(i,axis=0)
            #if df_trade_use.loc[i,'is_risk']==1:
            #    nofit_danger_count=nofit_danger_count+1

    print(nofit_count)
    #print(nofit_danger_count)
    return df_trade_use

In [24]:
df_trade_new=JD_trade_merge(df_log_use,df_trade_use)

23727


In [25]:
df_trade_new

,rowkey,id,is_risk,time_pd,weekdays,device,log_from,ip,city,result,timestamp,type,is_scan_num,is_sec_num
0,2,171909,0,2015-01-01 00:00:40,3,57192,2,363195,217,1,1.430000e+09,3,0,0
1,3,27863,0,2015-01-01 00:00:41,3,165080,2,901964,418,31,1.430000e+09,3,0,0
2,6,115201,0,2015-01-01 00:00:46,3,573956,1,691374,243,1,1.430000e+09,3,0,0
3,13,143711,0,2015-01-01 00:01:02,3,705591,10,607056,194,1,1.420000e+09,1,1,0
4,18,62643,0,2015-01-01 00:01:17,3,1043947,1,1060447,492,1,1.440000e+09,1,0,0
5,25,163830,0,2015-01-01 00:02:03,3,494882,2,1511481,194,1,1.430000e+09,3,0,0
6,32,172699,0,2015-01-01 00:03:00,3,98787,1,1137907,484,1,1.440000e+09,3,0,0
7,37,27863,0,2015-01-01 00:03:17,3,165080,2,901964,418,31,1.430000e+09,3,0,0
8,43,53265,0,2015-01-01 00:03:52,3,79302,2,1293102,364,1,1.430000e+09,2,0,0
9,52,72054,0,2015-01-01 00:04:22,3,856090,1,220194,488,1,1.440000e+09,3,0,0


In [ ]:
''' #a former version, could be used to clear the TEST data.
this_row='Null'
for i in range(len(df_trade_use)):
    df_tmp=df_log_use[df_log_use.loc[:,'id']==df_trade_use.loc[i,'id']]
    sigsig=False
    #this_row='Null'
    for rowd in df_tmp.iterrows():
        row=rowd[1]
        if df_trade_use.loc[i,'time_pd']-row['time_pd']<pd.Timedelta('01:00:00'):
            df_trade_use.loc[i,'device']=row['device']
            df_trade_use.loc[i,'log_from']=row['log_from']
            df_trade_use.loc[i,'ip']=row['ip']
            df_trade_use.loc[i,'city']=row['city']
            df_trade_use.loc[i,'result']=row['result']
            df_trade_use.loc[i,'timestamp']=row['timestamp']
            df_trade_use.loc[i,'type']=row['type']
            df_trade_use.loc[i,'is_scan_num']=row['is_scan_num']
            df_trade_use.loc[i,'is_sec_num']=row['is_sec_num']
            sigsig=True
            break
        this_row=row.copy()
        #print(this_row)
    if sigsig==False:
        df_trade_use.loc[i,'device']=this_row['device']
        df_trade_use.loc[i,'log_from']=this_row['log_from']
        df_trade_use.loc[i,'ip']=this_row['ip']
        df_trade_use.loc[i,'city']=this_row['city']
        df_trade_use.loc[i,'result']=this_row['result']
        df_trade_use.loc[i,'timestamp']=this_row['timestamp']
        df_trade_use.loc[i,'type']=this_row['type']
        df_trade_use.loc[i,'is_scan_num']=this_row['is_scan_num']
        df_trade_use.loc[i,'is_sec_num']=this_row['is_sec_num']
'''

In [26]:
df_trade_new.to_csv(path_or_buf=nowpath+'/trade_new.csv',sep=',')